In [41]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [42]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [43]:
tf.keras.backend.clear_session()

In [44]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [45]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,247.134
1,2003-2,0.756708,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,305.708
2,2003-3,0.756879,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,284.168
3,2003-4,0.757050,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,286.594
4,2003-5,0.757222,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,559.950


In [46]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.874645,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.890157,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.906034,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.919415,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [47]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [48]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
0,-1.917914,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.524203,-2.146709,-0.699400,-1.745399,-1.973963,-3.018821,-2.686146
1,-1.892160,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.508666,-2.125900,-0.706157,-1.725016,-1.923436,-2.934320,-2.623049
2,-1.866406,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.496577,-2.105031,-0.712914,-1.704633,-1.872910,-2.849818,-2.559951
3,-1.840653,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.496879,-2.100075,-0.719672,-1.684250,-1.822384,-2.765317,-2.496853
4,-1.814899,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.498597,-2.080078,-0.726429,-1.663867,-1.771858,-2.680816,-2.433756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.399834,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.832198,0.050588,1.519517,1.163052,-0.960847,-0.093961,0.627995
158,1.402752,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.824048,0.042723,1.476375,1.168317,-0.968515,-0.074705,0.638908
159,1.405671,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.815727,0.029152,1.433232,1.173583,-0.976183,-0.055448,0.649820
160,1.408589,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.810089,-0.007439,1.390090,1.178848,-0.983851,-0.036192,0.660733


In [49]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      249.680
1      219.561
2      304.641
3      262.564
4      299.209
        ...   
157    338.767
158    419.962
159    387.800
160    391.122
161    409.739
Name: Minas Gerais - Consumo de Cimento (t), Length: 162, dtype: float64

In [50]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [51]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [52]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Minas Gerais - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado
126,0.544198,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.135116,0.964210,1.008346,0.811298,0.287865,0.502913,0.863809
127,0.559144,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.150078,0.924638,1.051039,0.826912,0.237130,0.464014,0.848131
128,0.574090,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.162632,0.882104,1.093731,0.842525,0.186395,0.425115,0.832454
129,0.589036,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.166985,0.846537,1.136424,0.858138,0.135660,0.386216,0.816777
130,0.603982,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.145581,0.796933,1.179116,0.873751,0.084924,0.347317,0.801100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.315321,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.874645,0.774558,0.789659,1.149269,-0.772657,0.183801,0.694771
188,1.308842,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.890157,0.806353,0.784605,1.140354,-0.757425,0.173158,0.684584
189,1.302363,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.906034,0.844284,0.779552,1.131439,-0.742194,0.162516,0.674397
190,1.295885,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.919415,0.884141,0.774498,1.122523,-0.726963,0.151874,0.664209


In [53]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [54]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [55]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(144, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(72, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
#                         validation_split=0.07,
                        batch_size=36,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [56]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [57]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1704672341, 1672913700, 1392053742, 616632797, 2682321203, 2435672671, 2506865951, 928650777, 602551440, 2999003504, 1180905606, 3564787276, 891693144, 3427483473, 2211718694, 4241460560, 616534555, 2938368804, 1405422591, 2956723398, 3448711408, 3043089421, 3366456128, 129915873, 798473110, 2554823128, 2626371454, 3647806637, 380170051, 1559436323, 1679247824, 452515152, 1807710877, 1538048202, 2493251750, 1177541758, 774302030, 3484845594, 4158347535, 408098397, 1340380137, 3634827450, 1889423732, 1248095072, 770228668, 1488911865, 293049274, 3650858418, 740274463, 4028224391]


Step: 0 ___________________________________________
val_loss: 1286.3863525390625
winner_seed: 1704672341


Step: 1 ___________________________________________
val_loss: 1728.6448974609375


Step: 2 ___________________________________________
val_loss: 1298.03076171875


Step: 3 ___________________________________________
val_loss: 1543.116455078125


Step: 4 ___________________________________________
val_lo

In [58]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 4s 234ms/step - loss: 255113.4375 - val_loss: 59711.9766
Epoch 2/10000
4/4 [==============================] - 0s 16ms/step - loss: 39478.6641 - val_loss: 4587.2002
Epoch 3/10000
4/4 [==============================] - 0s 16ms/step - loss: 4840.2686 - val_loss: 4425.9126
Epoch 4/10000
4/4 [==============================] - 0s 15ms/step - loss: 4282.0610 - val_loss: 6118.0356
Epoch 5/10000
4/4 [==============================] - 0s 17ms/step - loss: 5062.4277 - val_loss: 3386.1335
Epoch 6/10000
4/4 [==============================] - 0s 17ms/step - loss: 3751.2864 - val_loss: 2371.3921
Epoch 7/10000
4/4 [==============================] - 0s 16ms/step - loss: 3839.7419 - val_loss: 2298.8057
Epoch 8/10000
4/4 [==============================] - 0s 16ms/step - loss: 3624.6401 - val_loss: 4343.4204
Epoch 9/10000
4/4 [==============================] - 0s 14ms/step - loss: 3584.8616 - val_loss: 4120.1636
Epoch 10/10000
4/4 [=====================

4/4 [==============================] - 0s 14ms/step - loss: 2469.5276 - val_loss: 1981.9263
Epoch 78/10000
4/4 [==============================] - 0s 14ms/step - loss: 2606.8694 - val_loss: 1827.5057
Epoch 79/10000
4/4 [==============================] - 0s 15ms/step - loss: 2275.7939 - val_loss: 1872.3160
Epoch 80/10000
4/4 [==============================] - 0s 15ms/step - loss: 2235.4580 - val_loss: 1799.1239
Epoch 81/10000
4/4 [==============================] - 0s 15ms/step - loss: 2241.3772 - val_loss: 1504.2047
Epoch 82/10000
4/4 [==============================] - 0s 15ms/step - loss: 2392.9958 - val_loss: 2628.7053
Epoch 83/10000
4/4 [==============================] - 0s 15ms/step - loss: 2367.5222 - val_loss: 4599.5630
Epoch 84/10000
4/4 [==============================] - 0s 15ms/step - loss: 2765.1431 - val_loss: 2097.9309
Epoch 85/10000
4/4 [==============================] - 0s 14ms/step - loss: 2477.2791 - val_loss: 2124.7217
Epoch 86/10000
4/4 [==============================] 

4/4 [==============================] - 0s 15ms/step - loss: 2424.1523 - val_loss: 2585.4949
Epoch 154/10000
4/4 [==============================] - 0s 16ms/step - loss: 2230.6118 - val_loss: 2729.1228
Epoch 155/10000
4/4 [==============================] - 0s 16ms/step - loss: 2527.6353 - val_loss: 1968.4446
Epoch 156/10000
4/4 [==============================] - 0s 16ms/step - loss: 2549.6292 - val_loss: 5449.4385
Epoch 157/10000
4/4 [==============================] - 0s 16ms/step - loss: 4218.1123 - val_loss: 2729.5405
Epoch 158/10000
4/4 [==============================] - 0s 14ms/step - loss: 2785.8018 - val_loss: 2062.8684
Epoch 159/10000
4/4 [==============================] - 0s 14ms/step - loss: 2505.0366 - val_loss: 2382.3447
Epoch 160/10000
4/4 [==============================] - 0s 14ms/step - loss: 2465.4202 - val_loss: 2841.0823
Epoch 161/10000
4/4 [==============================] - 0s 16ms/step - loss: 2234.7271 - val_loss: 2970.3354
Epoch 162/10000
4/4 [=======================

4/4 [==============================] - 0s 16ms/step - loss: 2101.6824 - val_loss: 2311.7100
Epoch 230/10000
4/4 [==============================] - 0s 16ms/step - loss: 2293.7241 - val_loss: 2301.0042
Epoch 231/10000
4/4 [==============================] - 0s 15ms/step - loss: 2375.6287 - val_loss: 4161.8467
Epoch 232/10000
4/4 [==============================] - 0s 16ms/step - loss: 2914.8206 - val_loss: 2628.4573
Epoch 233/10000
4/4 [==============================] - 0s 16ms/step - loss: 2353.0452 - val_loss: 2879.3230
Epoch 234/10000
4/4 [==============================] - 0s 16ms/step - loss: 2631.0896 - val_loss: 2221.4973
Epoch 235/10000
4/4 [==============================] - 0s 16ms/step - loss: 2204.9456 - val_loss: 1807.7351
Epoch 236/10000
4/4 [==============================] - 0s 17ms/step - loss: 2472.7642 - val_loss: 2071.7021
Epoch 237/10000
4/4 [==============================] - 0s 16ms/step - loss: 2164.7998 - val_loss: 2346.5859
Epoch 238/10000
4/4 [=======================

4/4 [==============================] - 0s 15ms/step - loss: 2020.0408 - val_loss: 1563.2184
Epoch 306/10000
4/4 [==============================] - 0s 16ms/step - loss: 1871.7997 - val_loss: 1395.2086
Epoch 307/10000
4/4 [==============================] - 0s 17ms/step - loss: 1907.6483 - val_loss: 1672.1206
Epoch 308/10000
4/4 [==============================] - 0s 15ms/step - loss: 2237.6475 - val_loss: 1755.2145
Epoch 309/10000
4/4 [==============================] - 0s 16ms/step - loss: 1828.5089 - val_loss: 1878.3527
Epoch 310/10000
4/4 [==============================] - 0s 14ms/step - loss: 1765.5911 - val_loss: 1591.0393
Epoch 311/10000
4/4 [==============================] - 0s 14ms/step - loss: 1974.3728 - val_loss: 1554.8612
Epoch 312/10000
4/4 [==============================] - 0s 14ms/step - loss: 2120.3728 - val_loss: 1883.4194
Epoch 313/10000
4/4 [==============================] - 0s 15ms/step - loss: 1905.3049 - val_loss: 1575.7748
Epoch 314/10000
4/4 [=======================

...

4/4 [==============================] - 0s 14ms/step - loss: 1450.9739 - val_loss: 2342.7410
Epoch 1590/10000
4/4 [==============================] - 0s 13ms/step - loss: 1204.9102 - val_loss: 2574.2134
Epoch 1591/10000
4/4 [==============================] - 0s 15ms/step - loss: 1581.5692 - val_loss: 2662.4724
Epoch 1592/10000
4/4 [==============================] - 0s 14ms/step - loss: 1303.0331 - val_loss: 2658.3906
Epoch 1593/10000
4/4 [==============================] - 0s 14ms/step - loss: 1309.1270 - val_loss: 2667.4001
Epoch 1594/10000
4/4 [==============================] - 0s 13ms/step - loss: 1399.9611 - val_loss: 2578.6709
Epoch 1595/10000
4/4 [==============================] - 0s 14ms/step - loss: 1490.8318 - val_loss: 2606.1899
Epoch 1596/10000
4/4 [==============================] - 0s 14ms/step - loss: 1330.8398 - val_loss: 2710.5496
Epoch 1597/10000
4/4 [==============================] - 0s 14ms/step - loss: 1349.7112 - val_loss: 2741.1030
Epoch 1598/10000
4/4 [==============

4/4 [==============================] - 0s 15ms/step - loss: 1692.4681 - val_loss: 2466.1938
Epoch 1665/10000
4/4 [==============================] - 0s 14ms/step - loss: 1764.7504 - val_loss: 2387.7974
Epoch 1666/10000
4/4 [==============================] - 0s 16ms/step - loss: 1721.4257 - val_loss: 2593.2578
Epoch 1667/10000
4/4 [==============================] - 0s 15ms/step - loss: 1706.3528 - val_loss: 2759.3872
Epoch 1668/10000
4/4 [==============================] - 0s 16ms/step - loss: 1752.7904 - val_loss: 2712.0134
Epoch 1669/10000
4/4 [==============================] - 0s 15ms/step - loss: 1493.7181 - val_loss: 2759.0906
Epoch 1670/10000
4/4 [==============================] - 0s 15ms/step - loss: 1760.6609 - val_loss: 2331.0032
Epoch 1671/10000
4/4 [==============================] - 0s 14ms/step - loss: 1827.6039 - val_loss: 2369.8267
Epoch 1672/10000
4/4 [==============================] - 0s 15ms/step - loss: 1357.8159 - val_loss: 2592.7468
Epoch 1673/10000
4/4 [==============

4/4 [==============================] - 0s 16ms/step - loss: 1649.6885 - val_loss: 1726.8153
Epoch 1740/10000
4/4 [==============================] - 0s 16ms/step - loss: 1494.1910 - val_loss: 1819.3008
Epoch 1741/10000
4/4 [==============================] - 0s 14ms/step - loss: 1587.3779 - val_loss: 1708.6658
Epoch 1742/10000
4/4 [==============================] - 0s 14ms/step - loss: 1457.3475 - val_loss: 1756.1100
Epoch 1743/10000
4/4 [==============================] - 0s 16ms/step - loss: 1375.8678 - val_loss: 1938.2880
Epoch 1744/10000
4/4 [==============================] - 0s 16ms/step - loss: 1457.4115 - val_loss: 2065.0630
Epoch 1745/10000
4/4 [==============================] - 0s 16ms/step - loss: 1484.0604 - val_loss: 1379.0531
Epoch 1746/10000
4/4 [==============================] - 0s 16ms/step - loss: 1210.4855 - val_loss: 1472.0474
Epoch 1747/10000
4/4 [==============================] - 0s 16ms/step - loss: 1302.4708 - val_loss: 1273.7650
Epoch 1748/10000
4/4 [==============

4/4 [==============================] - 0s 15ms/step - loss: 1587.6648 - val_loss: 1615.2806
Epoch 1815/10000
4/4 [==============================] - 0s 15ms/step - loss: 1341.5890 - val_loss: 1830.0248
Epoch 1816/10000
4/4 [==============================] - 0s 16ms/step - loss: 1577.3730 - val_loss: 1651.7709
Epoch 1817/10000
4/4 [==============================] - 0s 16ms/step - loss: 1377.3597 - val_loss: 1784.5747
Epoch 1818/10000
4/4 [==============================] - 0s 16ms/step - loss: 1523.9451 - val_loss: 2128.2583
Epoch 1819/10000
4/4 [==============================] - 0s 15ms/step - loss: 1809.6615 - val_loss: 1713.5721
Epoch 1820/10000
4/4 [==============================] - 0s 16ms/step - loss: 1411.5533 - val_loss: 1728.0604
Epoch 1821/10000
4/4 [==============================] - 0s 17ms/step - loss: 1318.9589 - val_loss: 1905.7848
Epoch 1822/10000
4/4 [==============================] - 0s 15ms/step - loss: 1496.2924 - val_loss: 1581.1561
Epoch 1823/10000
4/4 [==============

4/4 [==============================] - 0s 15ms/step - loss: 1396.7902 - val_loss: 1487.8921
Epoch 1890/10000
4/4 [==============================] - 0s 16ms/step - loss: 1345.7715 - val_loss: 1465.4832
Epoch 1891/10000
4/4 [==============================] - 0s 16ms/step - loss: 1453.3726 - val_loss: 1271.3727
Epoch 1892/10000
4/4 [==============================] - 0s 16ms/step - loss: 1492.9163 - val_loss: 1428.1857
Epoch 1893/10000
4/4 [==============================] - 0s 15ms/step - loss: 1406.9828 - val_loss: 1728.5370
Epoch 1894/10000
4/4 [==============================] - 0s 16ms/step - loss: 1335.3168 - val_loss: 2382.4766
Epoch 1895/10000
4/4 [==============================] - 0s 16ms/step - loss: 1349.7267 - val_loss: 2351.4563
Epoch 1896/10000
4/4 [==============================] - 0s 15ms/step - loss: 1366.6781 - val_loss: 2413.7544
Epoch 1897/10000
4/4 [==============================] - 0s 15ms/step - loss: 1342.9512 - val_loss: 2231.6338
Epoch 1898/10000
4/4 [==============

In [59]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [60]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 23ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,398.322815,398.313995,398.319458,398.335724,398.330505,399.342804,399.521149,399.199371,400.278381,399.600708,399.496155,399.552673,399.627045,399.399445,399.626221,399.630005,401.363068,399.626862,400.269775,406.97879,406.49173,405.112152,403.78479,407.151367,404.306427,407.457275,406.296326,406.15271,400.211151,404.690948
Target,413.46,497.019,476.478,421.395,413.641,338.033,431.028,305.105,402.406,419.419,336.054,568.395,541.488,537.851,526.265,559.293,466.646,416.049,501.489,460.699,445.583,509.271,532.602,500.905,588.495,589.349,543.982,614.074,557.419,412.582
Error,15.137177,98.705017,78.158539,23.059265,15.310486,61.309814,31.506866,94.09436,2.127625,19.818298,63.442169,168.842346,141.860931,138.451569,126.638794,159.663025,65.282928,16.42215,101.219238,53.720215,39.091278,104.158844,128.8172,93.753632,184.188568,181.891724,137.685669,207.921265,157.207855,7.891052


In [61]:
display(mae)
display(mape)

90.579254

0.17904328

In [62]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [63]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'